In [28]:
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from rpy2.robjects import r, pandas2ri
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [29]:
pandas2ri.activate()

In [30]:
from isaac.constants import BASIC_TRAINING_COLS, YOKED_TRAINING_COLS

In [31]:
def are_all_coordinates_within_bounds(trial):
    
    for coord in ["o1.x", "o2.x", "o3.x", "o4.x", "o1.y", "o2.y", "o3.y", "o4.y"]:
        if (trial[coord] < 0).any().any():
            return False
        
    for x_coord in ["o1.x", "o2.x", "o3.x", "o4.x"]:
        if (trial[x_coord] > 6).any().any():
            return False 
        
    for y_coord in ["o1.y", "o2.y", "o3.y", "o4.y"]:
        if (trial[y_coord] > 4).any().any():
            return False
    
    return True

def add_velocity_from_positions(trial):
    POSITION_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["x", "y"]]
    VELOCITY_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["vx", "vy"]]
    S_PER_FRAME = 1./60

    positions = trial[POSITION_COLS]
    shifted_positions = positions.shift(1).fillna(positions.iloc[0])
    trial[VELOCITY_COLS] = (shifted_positions - positions) / S_PER_FRAME

# Experiment 1 

In [32]:
rdata_path = "data/experiment1.rdata"
r['load'](rdata_path)

'df.l','df.w','ls.clips','ls.events'


In [33]:
all_trials = r['ls.clips']

is_active = (r['df.l'].condition == '1')
is_passive = (r['df.l'].condition == '0')

responses = r['df.l'][["trueMass", "trueRelationship"]]
YOKED_PATH = "data/yoked_trials_exp1.h5"
PASSIVE_PATH = "data/passive_trials_exp1.h5"

ALTERNATIVE_TRAINING_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["x", "y", "vx", "vy"]]

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [34]:
if os.path.exists(YOKED_PATH):
    os.remove(YOKED_PATH)

if os.path.exists(PASSIVE_PATH):
    os.remove(PASSIVE_PATH)

In [35]:
flat_trials = []
trial_i = 0

for participant in all_trials:
    for trial in participant:
        trial = pandas2ri.ri2py_dataframe(trial)
        # print(trial["idControlledObject"].nunique() > 1, is_active[trial_i])
        flat_trials.append(trial)
        trial_i += 1

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [36]:
len(flat_trials), len(is_active), len(is_passive), len(responses)

(484, 704, 704, 704)

In [37]:
is_active.sum(), is_passive.sum()

(220, 264)

In [38]:
# The first 484 are either active or passive
print((is_active[:-220] == ~is_passive[:-220]).all())

#The last 220 trials are neither active nor passive, therefore they are yoked
print((is_active[-220:] == is_passive[-220:]).all())

True
True


In [39]:
skipped = 0

for trial_i, (trial, is_active_trial) in tqdm(enumerate(zip(flat_trials, is_active)), total=len(flat_trials)):

    response = responses.iloc[trial_i]
    
    if not set(ALTERNATIVE_TRAINING_COLS).issubset(trial.columns):
        skipped += 1
        add_velocity_from_positions(trial)
        
    trial = trial.rename({alt_col: col for col, alt_col in 
                          zip(BASIC_TRAINING_COLS, ALTERNATIVE_TRAINING_COLS)}, axis="columns")

    if not are_all_coordinates_within_bounds(trial):
        print("Skipping trial because coordinates out of bounds")
        continue


    trial["C_none"] = (trial["idControlledObject"] == "none")
    trial["C_O1"] = (trial["idControlledObject"] == "object1")
    trial["C_O2"] = (trial["idControlledObject"] == "object2")
    trial["C_O3"] = (trial["idControlledObject"] == "object3")
    trial["C_O4"] = (trial["idControlledObject"] == "object4")

    trial = trial[list(YOKED_TRAINING_COLS) + ["idControlledObject"]].copy()

    true_mass = response["trueMass"]
    trial["A"] = (true_mass == "A")
    trial["B"] = (true_mass == "B")
    trial["same"] = (true_mass == "same")

    true_relationship = response["trueRelationship"]
    trial["attract"] = (true_relationship == "attract")
    trial["none"] = (true_relationship == "none")
    trial["repel"] = (true_relationship == "repel")

    if is_active_trial:
        hdf_path = YOKED_PATH
    else:
        hdf_path = PASSIVE_PATH
        
    trial.to_hdf(path_or_buf=hdf_path, key="trial_"+str(trial_i))

print(skipped / 11, "participants for which velocities weren't recorded")
skipped=0

  6%|▋         | 31/484 [00:00<00:07, 58.89it/s]

Skipping trial because coordinates out of bounds


 41%|████▏     | 200/484 [00:03<00:05, 53.97it/s]

Skipping trial because coordinates out of bounds


 72%|███████▏  | 348/484 [00:06<00:02, 46.11it/s]

...

Skipping trial because coordinates out of bounds


 93%|█████████▎| 448/484 [00:09<00:00, 37.92it/s]

Skipping trial because coordinates out of bounds


100%|██████████| 484/484 [00:10<00:00, 38.11it/s]

13.0 participants for which velocities weren't recorded
